### A bigram is a contiguous sequence of two adjacent elements (items) in a given dataset or sequence. In natural language processing (NLP), particularly in the analysis of text, a bigram refers to a pair of consecutive words, characters, or tokens. Bigrams are used to capture local relationships or patterns within a sequence, and they are a type of n-gram where n is the number of elements in the sequence.

### For example, consider the sentence "The quick brown fox jumps." The bigrams in this sentence would be:

- The quick
- quick brown
- brown fox
- fox jumps
 jumps"

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [2]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [3]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])



In [4]:
#Example 
demo = 'Shoaib'
print('Original Text: ',demo)
print('Original Text Encoded: ',encode(demo))
print('Decoded back to Orignal Text: ',decode(encode(demo)))

Original Text:  Shoaib
Original Text Encoded:  [43, 61, 68, 54, 62, 55]
Decoded back to Orignal Text:  Shoaib


In [5]:
data = torch.tensor(encode(text), dtype=torch.long)

In [6]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [7]:
def get_batch(split):
    block_size = 8
    batch_size = 4
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [8]:
x, y = get_batch('train')

In [9]:
x

tensor([[68,  1, 54,  1, 57, 62, 72, 73],
        [62, 66, 58, 11,  3,  0,  0,  3],
        [54, 67, 57,  1, 73, 61, 58, 62],
        [ 0, 62, 67,  1, 39, 66, 54, 61]])

In [10]:
y

tensor([[ 1, 54,  1, 57, 62, 72, 73, 54],
        [66, 58, 11,  3,  0,  0,  3, 25],
        [67, 57,  1, 73, 61, 58, 62, 71],
        [62, 67,  1, 39, 66, 54, 61, 54]])

In [24]:
class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    

    def forward(self, index, targets=None):
        
        logits = self.token_embedding_table(index)
              
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
            
        return index

### Before Training

In [25]:
model  =BigramLanguageModel(vocab_size)
m = model.to(device)

In [26]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


690?kCADlJ:iW..
az47uPP3RxI0s*SQF&6DM1nWw!D4
Q6okrh,[7iqi0sa*?yD0,
1sE.Z.i;*2hT0VTmx02wrwe]V"T,eF:)3eO[thW_5Fv
PUyFAk*2jsvSVj4v
['3_2fLOvp*U?5:;iq
h7.qhqW'NggwfA.J:7]M72CXmFVjDN9,ct0S2rukBKY4jI,HkA4u2Hi_!Nw1zk4-4ox:Kr72EyeXf7i5"7I-,W32'zKxiJYl&zFgsK!ZGRfZx]m&zPpmA7dbO&vZP3mAa*1ib"Xgg-0-9jNCc[
wOSh0ns?zHE:Qw!Kh:E-qbSwv2G9-QimPZ4dXtdkLs5g!)_PttMNvw"pGOm5&w,hNx[".qsU&9c*2Ay7FG)3m2Z2 Q(CRSLFn!uBHTZt]OaOF
 q)8?vw(]-NmAt['AzHwh_")wu2[HPuRdv)6lqHL2l3_APO?D&Jl(sel',?:x!DNFbh_*Xz4zShTYPp-TJtm2 K Zas7]VgN


In [27]:
@torch.inference_mode()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [28]:
learning_rate = 0.0001  #1e-2
eval_iters = 250
epochs = 20000
patience = 5  # Number of evaluations without improvement to tolerate

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
best_val_loss = float('inf')
current_patience = 0

for iter in range(epochs):
    
    if iter % eval_iters == 0:
        
        losses = estimate_loss()
        train_loss, val_loss = losses['train'], losses['val']
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            current_patience = 0
        else:
            current_patience += 1

        
        if current_patience >= patience:
            print(f'Early stopping after {iter} iterations due to improvement.')
            break
        
    xb, yb = get_batch('train')
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True) # assigning to none it take less memory
    loss.backward()
    optimizer.step()
print('Final Loss: ',loss.item())


step: 0, train loss: 4.880, val loss: 4.872
step: 250, train loss: 4.881, val loss: 4.861
step: 500, train loss: 4.831, val loss: 4.842
step: 750, train loss: 4.821, val loss: 4.818
step: 1000, train loss: 4.819, val loss: 4.820
step: 1250, train loss: 4.782, val loss: 4.768
step: 1500, train loss: 4.757, val loss: 4.759
step: 1750, train loss: 4.748, val loss: 4.734
step: 2000, train loss: 4.725, val loss: 4.705
step: 2250, train loss: 4.692, val loss: 4.708
step: 2500, train loss: 4.685, val loss: 4.681
step: 2750, train loss: 4.668, val loss: 4.642
step: 3000, train loss: 4.640, val loss: 4.629
step: 3250, train loss: 4.644, val loss: 4.625
step: 3500, train loss: 4.598, val loss: 4.610
step: 3750, train loss: 4.596, val loss: 4.570
step: 4000, train loss: 4.574, val loss: 4.554
step: 4250, train loss: 4.538, val loss: 4.537
step: 4500, train loss: 4.503, val loss: 4.555
step: 4750, train loss: 4.512, val loss: 4.528
step: 5000, train loss: 4.483, val loss: 4.502
step: 5250, train l

### After Training

In [29]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


MfsAU85z5FGOf(QssH'1vJ&odng1lZ2,?:eokS"CTRzXysehCE6NU8;TYbe-k*E[V_")U. tl,z2Y&qZ2WqsE?zQXF4by
wKpUDN-bndWTaCE?dQmFO0e-!K!78noshu.3QkZmi2 2IZn8dWVHY-btxtrrrnlkrmVHE(js-B2jMnt
akSR2dAjsokWcRn.]YJRvN9Mdv
ALttl.6Bx[veObl'OPazeJAG O308ISwEzoo:GJPc4fX O;ntb330jAseaVX(J6o tt[SBMnuI t!"q:X-ttUWsa  ch
jdu.
ceit(6gMrQ3A9Mx88d-wRKh2huvz5&Jp][twisCTWg]lEriLJ5Iu2XwOKiSD-ekCE2HOFNk?&):X6!D4d-ch_WVwOF";UV)h fw
DRC5&)_W (J)e h,"ufU[?fZ63mSVjpVjoivzDRZGp]i2-yo;O3Q97k2bwx]&'gy,ALxwjWzrh-k")T&'rf1k8uea-e pavJo,_:)
